# Project Outline

In [1]:
##########################################################################
# Imports (from openmm library)
##########################################################################
from __future__ import print_function
from simtk.openmm.app import *
from simtk.openmm import *
from simtk import unit
from simtk.openmm.app.pdbfile import *
import volumeEditor as ve

def Sys_Setup(pdb_file):
    """Sets up system for water simulation with given pdb file and minimizes"""
    ##########################################################################
    # System Setup
    ##########################################################################
    print('Loading Force Field and PDB file...')
    forcefield = app.ForceField('amoeba2013.xml')
    pdb = app.PDBFile(pdb_file)  # load to the pdb file

    # Creating the system
    print('Creating System...')
    system = forcefield.createSystem(pdb.topology,
        nonbondedMethod=app.NoCutoff, constraints=None, rigidWater=False)

    print('Creating Langevin Integrator...')
    integrator = LangevinIntegrator(200*unit.kelvin, 1.0/unit.picoseconds, 
        0.5*unit.femtoseconds)

    print('Setting Platform...')
    platform = Platform.getPlatformByName('CUDA')  # tell OpenMM to use the CUDA platform
    properties = {'CudaPrecision': 'mixed'}

    print('Creating Simulation')
    mySimulation = app.Simulation(pdb.topology, system, integrator, platform, properties)  # create the simulation object
    mySimulation.context.setPositions(pdb.positions)  # set the positions for the simulations based on our pdb object and file

    ##########################################################################
    # Minimize the system
    ##########################################################################
    print('Minimizing...')
    mySimulation.minimizeEnergy(maxIterations=1000)

    #PDBFile.writeFile(mySimulation.topology, pdb.positions, open('waterSphere.equilibrate.pdb', 'w'))



    """simulation.reporters.append(app.DCDReporter('waterSphere.dcd', 40))
    simulation.reporters.append(app.PDBReporter('waterSphere_trajectory.pdb', 40))
    simulation.reporters.append(app.StateDataReporter('WaterSphere.log', 40, step=True, 
        time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, 
        temperature=True, volume=True, density=True, progress=True, 
        remainingTime=True, speed=True, totalSteps=20000, separator='\t'))"""

    print('Running Production...')
    return mySimulation

def setanddwell(mySimulation,temp):
    """sets simulation (mySimulation) to deisred temperature(temp) and dwells at that 
    temperature for desired stepcounts (dwellcount)"""
    
    errorcount = 0
    maxerrors = 5
    Sim_Temp = Temp*kelvin
    Equlib_Steps = 500
    
    # Calculate Temperature Conversion. Used to calculate Temperature
    BOLTZ = MOLAR_GAS_CONSTANT_R
# calculate degrees of freedom in the system.  This used to calculate simulation temperature
    particles = mySimulation.system.getNumParticles()
    constraints = mySimulation.system.getNumConstraints()
    forces = [mySimulation.system.getForce(i) for i in range(mySimulation.system.getNumForces())]
    usesCMMR = any(isinstance(f, CMMotionRemover) for f in forces)
    DOF = 3*particles-constraints 
    if usesCMMR: DOF -= 3

# Set minimum error in sim temperature error reltive to set temperature
    Temp_tolerance = 0.0075

# Staring new simulation
    mySimulation.context.setTime(0)
    mySimulation.currentStep = 0
    mySimulation.integrator.setTemperature(Sim_Temp)

# Begin Cycle of testing simulation temperature to set point temperature
    Equlib_Cycle = 0
    Current_Temp=mySimulation.context.getState(getEnergy=True).getKineticEnergy()/(0.5*DOF*BOLTZ)
    Temp_Err = Current_Temp/Sim_Temp - 1

    while (abs(Temp_Err) > Temp_tolerance):
        Equlib_Cycle +=1
        print ("     Starting Thermal Equlib Step: {}".format(Equlib_Cycle))
        done=False
        dwell(mySimulation,Equlib_Steps)
        
        Current_Temp=mySimulation.context.getState(getEnergy=True).getKineticEnergy()/(0.5*DOF*BOLTZ)
        Temp_Err = Current_Temp/Sim_Temp - 1
        print("         Simulation T=","{:.1f}".format(Current_Temp/kelvin),"K.  Temp Error = ", "{:+.2f}".format(Temp_Err*100),"%")
    print ('    Equilibrated Temp = {:.1f}K'.format(Current_Temp/kelvin))
    print ('    Thermal Equilibrated for ', mySimulation.currentStep, ' steps')

    return mySimulation


def SimAtT(mySimulation,Temp,intervalTime,intervalCount,FileNameBase):
#Run the requested number of simulations at set temperature.  Save incremental .pdb files.  Replace .chk file

    errorcount = 0
    maxerrors = 5

    simTemp=Temp*kelvin
    WriteFile=FileNameBase + "{:.0f}".format(Temp)+"K"

# Reporting Step Definitions
    Trajectory_Step = intervalCount
    Log_Step= intervalCount
# Initialize Reporters
    mySimulation.reporters.append(PDBReporter(WriteFile + '_traj.pdb', Trajectory_Step))
    mySimulation.reporters.append(StateDataReporter(WriteFile + '_sim.log', Log_Step, step=True, 
        time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, 
        temperature=True,  progress=True, 
        remainingTime=True, speed=True, totalSteps=Simulation_steps*intervalCount, separator='\t'))

# Reset simulation
    mySimulation.context.setTime(0)
    mySimulation.currentStep = 0

# Start the iterated simulation!
    print('Running Temperature Trajectory: ', WriteFile)
    print('Startingtime: ',time.ctime())


    dwell(mySimulation,Equlib_Steps)
    if errorcount > maxerrors:
        break


  """   print("      Saving Data")
        with open(FileNameBase+'.chk','wb') as f:
            f.write(mySimulation.context.createCheckpoint())
        positions = mySimulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions()
        with open(FileNameBase+"_" +"{:0>2d}".format(i+1)+".pdb","w") as f:
            PDBFile.writeFile(mySimulation.topology, positions, f)
"""
#stop reporting progress and trajectory
    mySimulation.reporters.pop()
    mySimulation.reporters.pop()

    if errorcount<=maxerrors:
        print('Simulation Complete!')	
    else:
        print('Simulation '+WriteFile+ 'Failed.')

    return (errorcount<=maxerrors)
# End SimAtT




def dwell(mySimulation,Equlib_Steps):
    while errorcount <= maxerrors and not done:
            try:
                mySimulation.step(Equlib_Steps)
                done=True
            except:
                print('Simulation error in Thermal Equilibration call')
                traceback.print_exc()
                errorcount += 1
                if errorcount > maxerrors:
                    print('Maximum number of errors reached.  Terminating!')
                    sys.exit(1)
                else:
                    print('Resetting Simulation Temperature at ', time.ctime())
                    statePositions = mySimulation.context.getState(getPositions=True, enforcePeriodicBox=True)
                    mySimulation.context.reinitialize()
                    mySimulation.context.setState(statePositions)
                    mySimulation.context.setVelocitiesToTemperature(Sim_Temp,int(time.time()))
                    print('Restarting Thermal Equilibration')
    

IndentationError: expected an indented block (<ipython-input-1-df4297d22dc8>, line 137)

In [ ]:
def main():
    
    mySimulation = Sys_Setup(pdb_file)
    setanddwell(mySimulation,300,2000)

    for T in range(295,200,-5): # range is 295-200K in increments of -5K
        setanddwell(mySimulation,T)
        dwell(mySImulation,T,500)

    for T in range (200,450,50): #range 200-400K in increments of 50K
        setanddwell(mySimulation,T)
        dwell(mySimulation,1000)
        simAtT(mySimulation,T,4000,40,watersimulation)

    
    


#  Etracting values from simulation results

In [ ]:
import numpy as np
x=empty()

for i in range(100):
    data= np.loadtxt("waterframe{}.txt".format(i))
    identity=data[:,0]
    x,y,z=data[:,1],data[:,2],data[:,3]
    for i in len(x):
    
    
    
    